# Replicating Hierarchical Dirichlet Processes

## Abstract

The Hierarchical Dirichlet Process (HDP) is a Bayesian nonparametric algorithm which allows us to model the distribution of data (such as words) that come from unique groups (such as documents). The model assumes each observation within a specific group is drawn from a mixture model (such as a set of related topics), where we expect for there to be some variation and some overlap between mixture model components between groups. HDP is constructed by putting a set of hierarchical priors on a set of Dirichlet Processes. As a result, HDP can infer the number of mixture components across the data without any user specification beyond a choice in prior.

Below, we derive and implement two Markov chain Monte Carlo (MCMC) algorithms for the Hierarchical Dirichlet Process (HDP) with a Categorical-Dirichlet conjugate setup, as proposed by Teh, et al (2006): (1) an implementation of the Chinese Restaurant Franchise, and (2) a direct sampler based around the stick-breaking construction of the Dirichlet Process. We compare their run-time efficiency and output on simulated data, and thereafter apply the more efficient algorithm on scientific abstract data. We find that applying HDP to a set of abstracts results in a similar clustering scheme to optimized Latent Dirichlet Allocation. These results are consistent with the paper.

## Background

In statistics, there is often a desire to create models that express that the data is split into unique partitions, yet has the potential to have some partitions linked to each other. Hierarchical modeling is a natural Bayesian way to satisfy these desires; there are parameters that define between-group centering and variation, and there are parameters that express within-group centering and variation.

Simultaneously, while classical Bayesian statistics is excellent at expressing uncertainty by putting priors on all parameters in a model, assuming a finite amount of parameters can be a restrictive assumption on the data-generating mechanism for complex problems. Nonparametric Bayesian inference is the study of Bayesian algorithms that can have large support.

Model-based clustering problems are often inherently nonparametric in data-generation, yet there is often a desire to make inference on clustering output. Bayesian nonparametric algorithms balance these two stipulations and methods like the Dirichlet process and the Gaussian process are popular choices to satisfy these constraints.

Sometimes, however, we are interested in performing model-based clustering on grouped data, and in addition to making inference on the clustering output, we want for within-group clusters to potentially be shared between some groups. For instance, imagine we have genetic data with $k$ binary markers. While there are up to $2^k$ patterns that a human could express, within any population, we expect certain only a small subset of possible sequences, *haplotypes*, to actually be observed. Additionally, the haplotypes that produce the strongest genes to survive could vary depending on what part of the world the data comes from, and it may be of interest to see which haplotypes are common among distinct parts of the world (e.g., Asia, Europe, Africa). Here, we have distinct grouped-data (people from different regions of the world) and granular data that we desire to cluster (binary markers that we want to cluster into haplotypes). Hierarchical Dirichlet processes address this problem directly, and has been used to cluster genetic information. Other common use cases of HDP include speech recognition in low-resourced languages [https://arxiv.org/ftp/arxiv/papers/1606/1606.05967.pdf] and identifying abnormal behaviors in video in computer vision problems [https://arxiv.org/pdf/1606.08476.pdf].

Two potential downsides of Hierarchical Dirichlet Processes are that (1) as is the case with many Bayesian nonparametric methods, it is a complicated and computationally expensive way to handle clustering problems, and (2) among Bayesian nonparametric clustering methods, it may not always be the most appropriate (compared to say, the Pitman-Yor algorithm) due to being biased towards the "rich get richer" property, especially for physical and social science data that closely follow Zipf's law. However, the upsides are that it is an elegant, flexible, and self-sufficient algorithm to handle group-clustering problems with cluster sharing. 

In our own research, we are interested in another common use of HDP, topic modeling among documents, where each document can contain multiple topics, but we expect for there to be some topics that frequently occur throughout the entire set of documents. In this paper, we apply our implementation of HDP on documents, and compare that to other common methods of topic modeling.

## Description of Algorithm
The Hierarchical Dirichlet Process mixture model is given by
$$\begin{aligned}
G_0 | \gamma, H &\sim DP(\gamma, H) \\
G_j | \alpha_0, G_0 &\sim DP(\alpha_0, G_0) \\
\theta_{ji} | G_j &\sim G_j \\
x_{ji} | \theta_{ji} &\sim F(\theta_{ji})
\end{aligned} $$

This model is able to non-parametrically cluster each group's data while sharing information both between and within groups. $DP$ here represents Dirichlet process. There are a few illustrative ways to understand the Dirichlet process, and by extension, the Hierarchical Dirichlet process, and we briefly outline some of them below and connect them to HDP.

### Stick-breaking representation
A Dirichlet process is a measure on measures, and a random measure $P$ on $(\mathfrak{X}, \mathscr{X})$ is a Dirichlet process, $DP(\alpha)$, if for every finite measurable partition $A_1,...,A_k$ of $\mathfrak{X}$, 
$$
\begin{aligned}
(P(A_1),...,P(A_k)) \sim \textrm{Dirichlet}(k; \alpha(A_1),...,\alpha(A_k))
\end{aligned}
$$ 

It has been shown that a $DP$ can be represented as atoms drawn from a (not-necessarily discrete) base measure $H$ and gradually decreasing weights determined by the "stick-breaking process" (Sethuraman, 1994). The "stick-breaking process" is often represented by the Griffiths-Engen-McCloskey (GEM) distribution representing the "stick-breaking process", defined as:
$$
\begin{aligned}
w'_k &\overset{\textrm{i.i.d.}}{\sim} \textrm{Beta}(1, \alpha), k=1,2,... \\
w_k &:= w'_k \prod_{j<k}(1-w'_j)
\end{aligned}
$$
This is called a "stick-breaking process" because if we had a stick and we want to split it many times, $w'_k$ represents the proportion of the stick that has not yet been split off by time $k-1$ that is split at time point $k$, and $w_k$ represents the proportion of the stick that is split at time point $k$.

In the HDP, each group is a Dirichlet process drawn from another DP $G_0$, so these will contain the same atoms as $G_0$ but with different weights:
$$\begin{aligned}
G_0 &= \sum_{k=1}^{\infty} \beta_k \delta(\phi_k) \\
G_j &= \sum_{k=1}^{\infty} \pi_{jk} \delta(\phi_k) \\
\phi_k | H &\sim H
\end{aligned} $$
Additionally, if we define $\beta, \pi_j$ as the collected weights above, it can be shown that these vectors encode a distribution over $\mathbb{Z}^+$ such that $\beta | \gamma \sim GEM(\gamma)$ and $\pi_j | \alpha_0, \beta \sim DP(\alpha_0, \beta)$, where $GEM$ represents the Griffiths-Engen-McCloskey distribution. 

### Chinese restaurant/Blackwell-MacQueen urn representation
We can also express Dirichlet processes by how the data is partitioned. Successive draws from a DP exhibit clustering behavior, since the probability of taking a certain value is a related to the number of previous draws of that value. We can represent the distribution of the partition of observations into clusters with the *Chinese restaurant* process. Imagine a Chinese restaurant with the potential for an unbounded number of tables, each ordering one dish. The probability of person $\theta_i$ sits at a specific table $k \in 1,...,K$ can be represented by:
$$
\begin{aligned}
\theta_i \mid \theta_1,\theta_2,...\theta_{i-1},\alpha, G_0 \sim \sum_{k=1}^{K} \frac{m_{k}}{i - 1 + \alpha} \delta(\phi_k) +
                            \frac{\alpha}{i - 1 + \alpha} G_0
\end{aligned}
$$
Here, $m_k$ is the total number of customers $1,...,i-1$ sitting at table $k$, and $\alpha$ represents the probability of sitting at a completely unoccupied table up to that point. The key discovery that comes out of this representation is that a person is likeliest to sit down at tables with more people. This property is often called the "rich get richer" property.

The HDP extension is called the Chinese restaurant franchise process, where we have an unbounded number of restaurants, all with the same menu. Let $\phi_k$ be the global dishes, drawn from $H$; $\psi_{jt}$ be the table-specific dishes, drawn from $G_0$; and $\theta_{ji}$ be the customer-specific dishes, drawn from $G_j$.  Denote $z_{ji}$ as the dish index eaten by customer $ji$; $t_{ji}$ as the table index where customer $ji$ sits; $k_{jt}$ be the dish index served at table $jt$; $n_{jtk}$ be the customer counts; and $m_{jk}$ be the table counts. Then:

$$\begin{aligned}
\theta_{ji} | \text{other } \theta, \alpha_0, G_0 &\sim
    \sum_{t=1}^{m_{j\cdot}} \frac{n_{jt\cdot}}{i-1+\alpha_0} \delta(\psi_{jt}) +
                            \frac{\alpha_0}{i-1+\alpha_0} G_0 \\
\psi_{jt} | \text{other } \psi, \gamma, H &\sim
    \sum_{k=1}^{K} \frac{m_{\cdot k}}{m_{\cdot \cdot} + \gamma} \delta(\phi_k) +
                            \frac{\gamma}{m_{\cdot \cdot} + \gamma} H
\end{aligned} $$

We now walk through 2 Markov chain monte carlo algorithms based on different representations of Hierarchical Dirichlet processes, proposed in sections 5.1 and 5.3 of Teh, et al (2006).

### *( 5.1 )* Posterior Sampling in the Chinese Restaurant Franchise

Choose some base measure $h(\cdot)$ and a conjugate data-generating distribution $f(\cdot | \theta)$.  Important to compute are $f_k^{-x_{ji}}(x_{ji})$, the mixture component of customer $ij$ under $k$, and $f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$, the mixture component of table $jt$ under $k$.  This is done by integrating out $\phi_k$ over the joint density of all such points, for example:

$$\begin{aligned}
f_k^{-x_{ji}}(x_{ji}) &= \frac { \int f(x_{ij} | \phi_k) g(k)d\phi_k } { \int g(k)d\phi_k } \\
g(k) &= h(\phi_k) \prod_{j'i' \neq ji, z_{j'i'} = k} f(x_{j'i'} | \phi_k) 
\end{aligned} $$

The corresponding mixture components for a new customer assignment and new table assignment are denoted $f_{k^*}^{-x_{ji}}(x_{ji})$ and $f_{k^*}^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$, which are special cases of their the respective $f_k$ component where no data points have $z_{ij} = k^*$.

Using this, we first compute the likelihood of a given point $x_{ji}$ given the current clustering scheme:
$$
p(x_{ji} | t^{-ji}, t_{ji} = t^*, k) =
    \sum_{k=1}^{K} \frac{m_{\cdot k}}{m_{\cdot \cdot} + \gamma} f_k^{-x_{ji}}(x_{ji}) +
                            \frac{\gamma}{m_{\cdot \cdot} + \gamma} f_{k^*}^{-x_{ji}}(x_{ji})
$$

For efficiency, the Gibbs scheme implemented below only samples the $t$ and $k$ indexes (which can later be reverse-engineered to obtain the actual parameters).  The state space of the $k$ values is technically infinite, and the number of tables/dishes currently associated with the data is undefined.  We keep a running list of active $t$ and $k$ values.  Each update step, each customer is assigned either to one of the existing tables or to a new table, and if a customer is assigned to a new table, a new $k$ corresponding value gets drawn; similarly, each table is assigned a dish, either from the existing dishes or with a new dish.  If a table/dish becomes unrepresented in the current scheme, it gets removed from its respective list.  The update full conditionals are:

$$ \begin{aligned}
p(t_{ji} = t | t^{-ji}, k, ...) &\propto \begin{cases}
    n_{jt\cdot}^{-ji} f_{k_{jt}}^{-x_{ji}}(x_{ji}) & t\text{ used}\\
    \alpha_0 p(x_{ji} | ...) & t\text{ new}
    \end{cases} \\
p(k_{jt} = k | t, k^{-jt}) &\propto \begin{cases}
    m_{\cdot k} f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) & k\text{ used}\\
    \gamma f_{k^*}^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) & k\text{ new}
    \end{cases} \\
\end{aligned} $$

### *( 5.3 )* Posterior Sampling by Direct Assignment

This approach uses the same mixture components $f_k^{-x_{ji}}(x_{ji})$, but instead of sampling the $t$ then $k$ indexes, we directly sample $z_ji$ (the $k$ value for customer $ji$) and the count value $m_{jk}$ (the number of tables at restaurant $j$ serving $k$), and the weights $\beta$ for the $\phi_k$ atoms in the stick-breaking representation of $G_0$ given below:

$$ \begin{aligned}
\mathbf{\beta} &= (\beta_1, ..., \beta_K, \beta_u) \sim Dir(m_{\cdot 1}, ... m_{\cdot K}, \gamma) \\
G_u &\sim DP(\gamma, H) \\
p(\phi_k | t, k) &\sim h(\phi_k) \prod_{ji : k_{jt_{ji}} = k} f(x_{ji}| \phi_k) \\
G_0 &= \sum_{k=1}^K \beta_k \delta(\phi_k) + \beta_u G_u
\end{aligned} $$

The direct sampling simplifies bookkeeping slightly, as there is no need to keep an explicit map between table $jt$ and its corresponding dish $k$ (this is implied by the count $m_{jk}$ and the weight $\beta_k$).  Additionally, it greatly cuts down on the computation cost.  We still must loop through each customer sequentially and compute the customer-specific mixture components, but there is no longer a need to compute the table-specific components $f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$ when assigning the $k$ values.  The update full conditionals are:

$$ \begin{aligned}
p(z_{ji} = k | z^{-ji}, m, \beta) &= \begin{cases}
    (n_{j \cdot k}^{-ji} + \alpha_0 \beta_k) f_k^{x_{ji}}(x_{ji}) & k\text{ used}\\
    \alpha_0 \beta_u f_{k^*}^{-x_{ji}}(x_{ji}) & k\text{ new}
    \end{cases} \\
p(m_{jk} = m | z, m^{-jk}, \beta) &=
    \frac{\Gamma(\alpha_0 \beta_k)}{\Gamma(\alpha_0 \beta_k + n_{j \cdot k})}
    s(n_{j \cdot k}, m) (\alpha_0 \beta_k)^m \\
\mathbf{\beta} | m &\sim Dir(m_{\cdot 1}, ... m_{\cdot K}, \gamma)
\end{aligned} $$

If a dish $k$ becomes unused, its corresponding $\beta_k$ will get temporarily set to 0.  But if that same value reappears, we must redistribute the weights by setting $\beta_k' = b\beta_u$ and $\beta_u' = (1-b)\beta_u$, where $b \sim Beta(1, \gamma)$.

### Distribution-Specific Mixture Components

The only part of these sampling algorithms that depend on the choice of the measures $H$ and $F$ are the mixture components $f_k$, so this is the only part that needs rewritten for each type of model.  Let
$$ \begin{aligned}
V_{kji} &= \{ j'i' : j'i' \neq ji, z_{j'i'} = k \} \\
W_{kjt} &= \{ j'i' : j't_{j'i'} \neq jt, k_{j't_{j'i'}} = k \} \\
T_{jt} &= \{ j'i': t_{j'i'} = jt \} \\
\end{aligned} $$
$V$ is the set of all customers (excluding customer $ij$) eating dish $k$; $W$ is the set of all customers at tables (excluding table $jt$) eating $k$; these correspond to the product terms in the mixture components.  By conjugacy rules and kernel tricks, each $f_k$ can be expressed as functions of these sets.  Each $f_{k^*}$ can be found by using the corresponding $f_k$ formula where $V$ or $W$ is the empty set. Below we derive $f_k$ for a few important conjugate prior-likelihood combinations.

#### F = Poisson, H = Gamma

$$ \begin{aligned}
f(x | \phi_k) &\sim Poisson(\phi_k) \\
h(\phi_k) &\sim Gamma(\alpha, \beta) \\
\\
f_k^{-x_{ji}}(x_{ji}) &= \frac{1}{x_{ji}!} \cdot
    \frac{\Gamma(x_{ji} + \alpha_v)}{(1 + \beta_v)^{x_{ji} + \alpha_v}} \cdot
    \frac{(\beta_v)^{\alpha_v}}{\Gamma(\alpha_v)} \\
f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) &= \frac{1}{\prod_T x_t!} \cdot
    \frac{\Gamma(\sum_T x_t + \alpha_w)}{(|T| + \beta_w)^{\sum_T x_t + \alpha_w}} \cdot
    \frac{(\beta_w)^{\alpha_w}}{\Gamma(\alpha_w)} \\
\alpha_v &= \sum_V x_v + \alpha \quad , \quad \beta_v = |V| + \beta \\
\alpha_w &= \sum_W x_w + \alpha \quad , \quad \beta_w = |W| + \beta \\
\end{aligned} $$

#### F = Multinomial, H = Dirichlet

Let $\mathbf{x}$ be a feature vector of length $L$.  The Multinomial/Dirichlet model is given by
$$ \begin{aligned}
f(\mathbf{x} | n, \mathbf{\phi}_k) &\sim Multinomial(n, \mathbf{\phi}_k) \\
h(\mathbf{\phi}_k) &\sim Dirichlet(L, \mathbf{\alpha}) \\
\end{aligned} $$
Note that each $\mathbf{x}_{ji}$ can have a different value of $n_{ji}$, representing different size draws from the multinomial distribution.  But $\phi_k$, representing the relative concentrations of the $L$ components, will be the same.

$$ \begin{aligned}
f_k^{-\mathbf{x}_{ji}}(\mathbf{x}_{ji}) &= \frac{n_{ji}!}{\prod_{\ell=1}^L (\mathbf{x}_{ji})_\ell!} \cdot
    \frac{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) } \cdot
    \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) }{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
\mathbf{\alpha}_{\ell}^{bottom} &= \sum_V (\mathbf{x}_v)_{\ell} + \mathbf{\alpha}_{\ell} \\
\mathbf{\alpha}_{\ell}^{top} &= (\mathbf{x}_{ji})_{\ell} + \mathbf{\alpha}_{\ell}^{bottom} \\
\\
f_k^{-\mathbf{X}_{jt}}(\mathbf{X}_{jt}) &=
    \frac{ \prod_T n_t! }{ \prod_T \prod_{\ell=1}^L (\mathbf{x}_t)_\ell! } \cdot
    \frac{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) } \cdot
    \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) }{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
\mathbf{\alpha}_{\ell}^{bottom} &= \sum_W (\mathbf{x}_w)_{\ell} + \mathbf{\alpha}_{\ell} \\
\mathbf{\alpha}_{\ell}^{top} &= \sum_T (\mathbf{x}_t)_\ell + \mathbf{\alpha}_{\ell}^{bottom} \\
\end{aligned} $$

In the application of latent topic modeling for NLP, restaurant $j$ represents document $j$, and customer $ji$ represents some subset of the word counts in document $j$, where $L$ is the size of the entire vocabulary for the corpus of documents.  If a customer is one word, $\mathbf{x}_{ji}$ is a draw from a categorical distribution of size $L$, a special case of the multinomial; but a customer can also represent the set of all instances of a unique word, the set of words in a sentence, the set of words in a paragraph, etc.

The dishes $k$ represent topics (where $\mathbf{\phi}_k$ is a distribution over the vocabulary), and the tables $t$ represent clusters of words, sentences, paragraphs (however a "customer" is encoded) from a single document.  Due to the nature of the HDP, these topics are shared among documents and among clusters within a document.  If the customer subsets are mutually exclusive, then each section of a document will be assigned to exactly one topic; but if the customers overlap, the same section of a document could be assigned to multiple topics.

#### F = Categorical, H = Dirichlet

This is a special case of the above, where each vector $\mathbf{x} \sim Categorical(L, \mathbf{\phi}_k)$ (i.e., every entry is a zero except for a one in one position).  If we model $f$ in this way, the mixture components can be computed very efficiently.  Let $\mathbf{x}_{ji}$ have a one in position $\ell'$, and note that in this case

$$ \begin{aligned}
f_k^{-\mathbf{x}_{ji}}(\mathbf{x}_{ji}) &= \frac{1!}{1! \prod_{\ell \neq \ell'} 0!} \cdot
    \left[ \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) }
                { \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) } \right]^{-1}
    \prod \frac{ \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
    &= \left[ \frac{ \Gamma(1 + |V| + \sum \alpha_{\ell}) }
                   { \Gamma(|V| + \sum \alpha_{\ell}) } \right]^{-1}
    \prod \frac{ \Gamma((\mathbf{x}_{ji})_{\ell} + \mathbf{\alpha}_{\ell}^{bottom}) }
               { \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
    &= \left[ \frac{ \Gamma(1 + |V| + \sum \alpha_{\ell}) }
                   { \Gamma(|V| + \sum \alpha_{\ell}) } \right]^{-1}
       \frac{ \Gamma(1 + \mathbf{\alpha}_{\ell'}^{bottom}) }
            { \Gamma(\mathbf{\alpha}_{\ell'}^{bottom}) } \\
    &= [|V| + \sum \alpha_{\ell}]^{-1} \cdot \alpha_{\ell'}^{bottom} \\
    &= \frac{|\{v \in V : (x_v)_{\ell'} = 1\}| + \alpha_{\ell'}}
            {|V| + \sum \alpha_{\ell}}
\end{aligned} $$

Similarly, the table-specific mixture component is

$$ \begin{aligned}
f_k^{-\mathbf{X}_{jt}}(\mathbf{X}_{jt})
    &= \left[ \frac{ \Gamma(|T| + |V| + \sum \alpha_{\ell}) }
                   { \Gamma(|V| + \sum \alpha_{\ell}) } \right]^{-1}
       \prod_{\ell' \in \mathcal{L}}
       \frac{ \Gamma( \sum_T (\mathbf{x}_t)_{\ell'} + \sum_W (\mathbf{x}_w)_{\ell'} + \mathbf{\alpha}_{\ell'} ) }
            { \Gamma( \sum_W (\mathbf{x}_w)_{\ell'} + \mathbf{\alpha}_{\ell'} ) } \\
\end{aligned} $$

where $\mathcal{L} = \{\ell' : (x_t)_{\ell'} = 1 \text{ for some } t \in T \}$.  This does not simplify as nicely and remains somewhat computationally intensive, so the direct sampler has a major advantage with this type of model.